# person data 축소

In [10]:
import os
import random
import shutil

# 원본 person 데이터 경로
person_img_dir = "/home/usou/Downloads/small_dataset/train/images"
person_lbl_dir = "/home/usou/Downloads/small_dataset/train/labels"

# 분리 저장 경로
base_dir = "/home/usou/Downloads/800_person_dataset"
train_img_dir = os.path.join(base_dir, "train/images")
train_lbl_dir = os.path.join(base_dir, "train/labels")
valid_img_dir = os.path.join(base_dir, "valid/images")
valid_lbl_dir = os.path.join(base_dir, "valid/labels")

# 디렉토리 생성
for d in [train_img_dir, train_lbl_dir, valid_img_dir, valid_lbl_dir]:
    os.makedirs(d, exist_ok=True)

# 무작위 800장 추출
img_files = [f for f in os.listdir(person_img_dir) if f.endswith('.jpg')]
selected_imgs = random.sample(img_files, 800)

# 학습/검증 비율 나눔
train_imgs = selected_imgs[:600]
valid_imgs = selected_imgs[600:]

def copy_and_fix_label(img_list, src_img_dir, src_lbl_dir, dst_img_dir, dst_lbl_dir):
    copied = 0
    for img in img_list:
        lbl = img.replace('.jpg', '.txt')
        src_img = os.path.join(src_img_dir, img)
        src_lbl = os.path.join(src_lbl_dir, lbl)
        dst_img = os.path.join(dst_img_dir, img)
        dst_lbl = os.path.join(dst_lbl_dir, lbl)

        if os.path.exists(src_lbl):
            shutil.copy(src_img, dst_img)

            # 클래스 인덱스 0으로 고정
            with open(src_lbl, "r") as f:
                lines = f.readlines()
            fixed_lines = []
            for line in lines:
                parts = line.strip().split()
                if parts:
                    parts[0] = '0'  # person 클래스 인덱스 고정
                    fixed_lines.append(" ".join(parts))
            with open(dst_lbl, "w") as f:
                f.write("\n".join(fixed_lines) + "\n")
            copied += 1
        else:
            print(f"⚠️ 라벨 없음: {lbl}")
    return copied

# 복사 실행
train_count = copy_and_fix_label(train_imgs, person_img_dir, person_lbl_dir, train_img_dir, train_lbl_dir)
valid_count = copy_and_fix_label(valid_imgs, person_img_dir, person_lbl_dir, valid_img_dir, valid_lbl_dir)

print(f"✅ 완료: train {train_count}개, valid {valid_count}개")


✅ 완료: train 600개, valid 200개


In [11]:
import os
import shutil
import random

# 원본 roscar 구조
roscar_root = "/home/usou/Downloads/roscar_dataset"

# 타겟 위치
train_img_dst = "/home/usou/Downloads/800_person_dataset/train/images"
train_lbl_dst = "/home/usou/Downloads/800_person_dataset/train/labels"
valid_img_dst = "/home/usou/Downloads/800_person_dataset/valid/images"
valid_lbl_dst = "/home/usou/Downloads/800_person_dataset/valid/labels"

# 디렉토리 생성
for d in [train_img_dst, train_lbl_dst, valid_img_dst, valid_lbl_dst]:
    os.makedirs(d, exist_ok=True)

# 모든 하위 폴더 순회하며 roscar 이미지 수집
roscar_img_list = []

for subdir in os.listdir(roscar_root):
    subpath = os.path.join(roscar_root, subdir)
    if os.path.isdir(subpath):
        for file in os.listdir(subpath):
            if file.endswith('.jpg'):
                full_img_path = os.path.join(subpath, file)
                full_lbl_path = full_img_path.replace('.jpg', '.txt')
                if os.path.exists(full_lbl_path):
                    roscar_img_list.append((full_img_path, full_lbl_path))

# 무작위 섞기
random.shuffle(roscar_img_list)

# 600 train / 200 valid 분할
train_split = roscar_img_list[:600]
valid_split = roscar_img_list[600:800]

def copy_pair(pairs, img_dst, lbl_dst):
    for img_path, lbl_path in pairs:
        img_name = os.path.basename(img_path)
        lbl_name = os.path.basename(lbl_path)
        shutil.copy(img_path, os.path.join(img_dst, img_name))
        shutil.copy(lbl_path, os.path.join(lbl_dst, lbl_name))

copy_pair(train_split, train_img_dst, train_lbl_dst)
copy_pair(valid_split, valid_img_dst, valid_lbl_dst)

print("✅ roscar 데이터 600 train / 200 valid 분할 완료")


✅ roscar 데이터 600 train / 200 valid 분할 완료


# 모든걸 person으로 인식하는 문제 발생
    - .txt 라벨링 데이터에서 클래스 번호가 0이 person이라 1로 변경해야됨

In [12]:
import os

# roscar 라벨이 들어 있는 경로
label_dir = "/home/usou/Downloads/800_person_dataset/train/labels"

# 'roscar' 라벨링된 파일들이 특정 규칙 (예: 이름 패턴) 없을 경우 전부 처리
for fname in os.listdir(label_dir):
    if fname.endswith(".txt"):
        file_path = os.path.join(label_dir, fname)

        # 내용 읽고 첫 인덱스가 '0'이면 '1'로 바꾸기
        with open(file_path, "r") as f:
            lines = f.readlines()

        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if parts and parts[0] == '0':
                parts[0] = '1'
            new_lines.append(" ".join(parts) + "\n")

        # 파일 덮어쓰기
        with open(file_path, "w") as f:
            f.writelines(new_lines)

print("✅ 모든 .txt 라벨에서 클래스 0 → 1 변경 완료")


✅ 모든 .txt 라벨에서 클래스 0 → 1 변경 완료


In [14]:
import os

# roscar 라벨이 들어 있는 경로
label_dir = "/home/usou/Downloads/800_person_dataset/valid/labels"

# 'roscar' 라벨링된 파일들이 특정 규칙 (예: 이름 패턴) 없을 경우 전부 처리
for fname in os.listdir(label_dir):
    if fname.endswith(".txt"):
        file_path = os.path.join(label_dir, fname)

        # 내용 읽고 첫 인덱스가 '0'이면 '1'로 바꾸기
        with open(file_path, "r") as f:
            lines = f.readlines()

        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if parts and parts[0] == '0':
                parts[0] = '1'
            new_lines.append(" ".join(parts) + "\n")

        # 파일 덮어쓰기
        with open(file_path, "w") as f:
            f.writelines(new_lines)

print("✅ 모든 .txt 라벨에서 클래스 0 → 1 변경 완료")


✅ 모든 .txt 라벨에서 클래스 0 → 1 변경 완료


# 테스트

In [ ]:
import os

# roscar 라벨이 들어 있는 경로
label_dir = "/home/usou/Downloads/800_person_dataset/train/labels"

# 'roscar' 라벨링된 파일들이 특정 규칙 (예: 이름 패턴) 없을 경우 전부 처리
for fname in os.listdir(label_dir):
    if fname.endswith(".txt"):
        file_path = os.path.join(label_dir, fname)

        # 내용 읽고 첫 인덱스가 '0'이면 '1'로 바꾸기
        with open(file_path, "r") as f:
            lines = f.readlines()

        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if parts and parts[0] == '0':
                parts[0] = '1'
            new_lines.append(" ".join(parts) + "\n")

        # 파일 덮어쓰기
        with open(file_path, "w") as f:
            f.writelines(new_lines)

print("✅ 모든 .txt 라벨에서 클래스 0 → 1 변경 완료")


In [18]:
import cv2
from ultralytics import YOLO

# 1. 모델 로드
model = YOLO("/home/usou/dev_ws/final_yolo_test/src/runs/detect/train8(person+roscar)/weights/best.pt")

# 2. 웹캠 열기 (기본 카메라: 0)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ 웹캠을 열 수 없습니다.")
    exit()

print("✅ 웹캠 실행 중... ESC를 눌러 종료합니다.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ 프레임 수신 실패")
        break

    # 3. YOLO 추론
    results = model(frame, verbose=False)

    # 4. 결과 시각화
    annotated_frame = results[0].plot()

    # 5. 결과 출력
    cv2.imshow("YOLO Webcam Detection", annotated_frame)

    # 6. ESC 키(27번) 누르면 종료
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()


✅ 웹캠 실행 중... ESC를 눌러 종료합니다.
